## Problem

In this project I look to understand the predictability of a movie's rating based on some of the most basic factors like runtime, genre, directors. We will be looking at a large number of movies and seeing how we can model the ratings. To do this, we will be using IMDb's dataset on basic info, titles and ratings. 

Disclaimer: I personally think that such a model can only get you the basic trends and do not explain the full depth of the movie making process.

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data preprocessing

### a. Cleaning Titles dataset

In [2]:
titles = pd.read_csv('Titles.tsv', sep='\t')
titles.head()

C:\Users\zbtin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


1. Checking for null values

In [3]:
titles.title.isna().value_counts()

False    22543552
True            2
Name: title, dtype: int64

In [4]:
titles.titleId.isna().value_counts()

False    22543554
Name: titleId, dtype: int64

In [5]:
titles.region.isna().value_counts()

False    22543465
True           89
Name: region, dtype: int64

2. Removing null values

In [6]:
titles.dropna(inplace = True)

In [7]:
titles.shape

(22543463, 8)

In [8]:
titles.isOriginalTitle.value_counts()

0     13058629
0      9008568
1       374061
1       100070
\N        2135
Name: isOriginalTitle, dtype: int64

3. Standardizing the data in the isOriginalTitle Column

In [9]:
titles['isOriginalTitle'].replace('0', 0, inplace = True)
titles['isOriginalTitle'].replace('1', 1, inplace = True)
titles.isOriginalTitle.value_counts()

0     22067197
1       474131
\N        2135
Name: isOriginalTitle, dtype: int64

In [10]:
titles.region.value_counts().head(20)

FR     2789011
JP     2774882
DE     2743378
ES     2724525
IT     2711374
IN     2690382
PT     2662052
US     1060544
\N      533292
GB      277483
CA      133900
XWW     127899
BR       83975
AU       83058
GR       72032
MX       70984
FI       65493
RU       63537
HU       58550
PL       56332
Name: region, dtype: int64

Chloreopleth map can be created using this

In [11]:
titles.types.value_counts()

\N                      20893381
imdbDisplay               996495
original                  474132
alternative                70948
working                    44147
dvd                        19918
tv                         15707
video                      15478
festival                   12787
dvdimdbDisplay              232
imdbDisplaytv                70
festivalimdbDisplay          68
imdbDisplayworking           31
imdbDisplayvideo             25
alternativetv                 9
tvvideo                       6
alternativeworking            6
alternativedvd                6
videoworking                  5
tvworking                     4
festivalworking               3
alternativevideo              2
dvdvideo                      1
alternativefestival           1
dvdworking                    1
Name: types, dtype: int64

In [12]:
titles.language.value_counts().head(20)

\N     3830471
ja     2679053
fr     2644268
hi     2617721
es     2614816
de     2614694
it     2613618
pt     2613467
en      196422
ru       25468
bg       20587
tr       20066
cmn       9931
sv        7633
he        6412
qbn       4939
sr        3966
yue       3810
fa        2387
ca        2084
Name: language, dtype: int64

In [13]:
titles.attributes.value_counts()

\N                                   22350358
transliterated title                    24000
alternative spelling                    13215
literal English title                   12374
new title                               10512
                                       ...   
cable TV titlecut version                  1
IMAX versionpromotional title              1
fourth season titlerecut version           1
fortieth season title                       1
first two episodes title                    1
Name: attributes, Length: 188, dtype: int64

Since language, attributes and type have a predominance of null values we will not be using them to estimate the ratings later

In [14]:
titles.drop(columns=['attributes', 'language', 'types'], inplace = True)

In [15]:
titles.shape

(22543463, 5)

### Cleaning of Ratings dataset 

In [16]:
ratings = pd.read_csv('Ratings.tsv', sep='\t')
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1639
1,tt0000002,6.1,198
2,tt0000003,6.5,1331
3,tt0000004,6.2,120
4,tt0000005,6.1,2114


In [17]:
ratings.shape

(1059926, 3)

### Combining the two datasets

In [18]:
df = ratings.set_index('tconst').join(titles.set_index('titleId'), how='left')

In [19]:
df.shape

(2983481, 6)

In [20]:
df

,averageRating,numVotes,ordering,title,region,isOriginalTitle
tt0000001,5.6,1639,1.0,Карменсіта,UA,0
tt0000001,5.6,1639,2.0,Carmencita,DE,0
tt0000001,5.6,1639,3.0,Carmencita - spanyol tánc,HU,0
tt0000001,5.6,1639,4.0,Καρμενσίτα,GR,0
tt0000001,5.6,1639,5.0,Карменсита,RU,0
...,...,...,...,...,...,...
tt9916766,6.9,14,4.0,Folge #10.15,DE,0
tt9916766,6.9,14,5.0,Episódio #10.15,PT,0
tt9916766,6.9,14,6.0,एपिसोड #10.15,IN,0
tt9916766,6.9,14,7.0,Épisode #10.15,FR,0


We notice that the many movies have multiple entries, just diffenent title and region

Let's see which region has the most analyzable movies

In [21]:
df.region.value_counts()

US    390413
\N    326489
FR    145416
JP    133590
DE    128584
       ...  
VC         1
VG         1
ST         1
NR         1
SB         1
Name: region, Length: 230, dtype: int64

Given the news IMDb contains it is not unexpected that the US region will dominate

We will concentrate on the movies based in the US as this has the largest number of movies that have a rating associated with them

In [22]:
US_region = df['region']=='US'
df = df[US_region]
df

,averageRating,numVotes,ordering,title,region,isOriginalTitle
tt0000001,5.6,1639,6.0,Carmencita,US,0
tt0000002,6.1,198,7.0,The Clown and His Dogs,US,0
tt0000005,6.1,2114,1.0,Blacksmithing Scene,US,0
tt0000005,6.1,2114,5.0,Blacksmith Scene #1,US,0
tt0000005,6.1,2114,6.0,Blacksmithing,US,0
...,...,...,...,...,...,...
tt9916200,8.3,158,1.0,The Great Experiment,US,0
tt9916204,8.1,183,1.0,Better Angels,US,0
tt9916348,9.3,13,1.0,Ancient World Exposed,US,0
tt9916720,6.1,56,10.0,The Demonic Nun,US,0


Now we have an analyzable dataset, but ordering isn't telling us anything so let's remove it

In [23]:
df.drop(columns=['ordering'], inplace= True)
df

C:\Users\zbtin\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,averageRating,numVotes,title,region,isOriginalTitle
tt0000001,5.6,1639,Carmencita,US,0
tt0000002,6.1,198,The Clown and His Dogs,US,0
tt0000005,6.1,2114,Blacksmithing Scene,US,0
tt0000005,6.1,2114,Blacksmith Scene #1,US,0
tt0000005,6.1,2114,Blacksmithing,US,0
...,...,...,...,...,...
tt9916200,8.3,158,The Great Experiment,US,0
tt9916204,8.1,183,Better Angels,US,0
tt9916348,9.3,13,Ancient World Exposed,US,0
tt9916720,6.1,56,The Demonic Nun,US,0


### Cleaning basic info dataset and combining it with the others

In [24]:
basics = pd.read_csv('Basics.tsv', sep='\t')
basics.head()

C:\Users\zbtin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [25]:
basics.shape

(7041952, 9)

In [26]:
df = df.join(basics.set_index('tconst'), how='left')
df

,averageRating,numVotes,title,region,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,5.6,1639,Carmencita,US,0,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,6.1,198,The Clown and His Dogs,US,0,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000005,6.1,2114,Blacksmithing Scene,US,0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000005,6.1,2114,Blacksmith Scene #1,US,0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000005,6.1,2114,Blacksmithing,US,0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9916200,8.3,158,The Great Experiment,US,0,tvEpisode,The Great Experiment,The Great Experiment,0,2019,\N,43,Drama
tt9916204,8.1,183,Better Angels,US,0,tvEpisode,Better Angels,Better Angels,0,2019,\N,42,Drama
tt9916348,9.3,13,Ancient World Exposed,US,0,video,Ancient World Exposed,Ancient World Exposed,0,2019,\N,67,History
tt9916720,6.1,56,The Demonic Nun,US,0,short,The Nun 2,The Nun 2,0,2019,\N,10,"Comedy,Horror,Mystery"


In [27]:
df.titleType.value_counts()

movie           121613
tvEpisode        98318
short            59443
video            41774
tvSeries         30642
tvMovie          18723
tvSpecial         7411
videoGame         7315
tvMiniSeries      3312
tvShort           1862
Name: titleType, dtype: int64

We will only be looking at movies, as this is the dominant section of the data

In [28]:
movies = df['titleType']=='movie'
df = df[movies]
df

,averageRating,numVotes,title,region,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000009,5.9,154,Miss Jerry,US,0,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
tt0000147,5.2,355,The Corbett-Fitzsimmons Fight,US,0,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
tt0000630,3.2,12,Hamlet,US,0,movie,Hamlet,Amleto,0,1908,\N,\N,Drama
tt0000679,5.2,37,The Fairylogue and Radio-Plays,US,0,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,\N,120,"Adventure,Fantasy"
tt0000886,5.1,21,"Hamlet, Prince of Denmark",US,0,movie,"Hamlet, Prince of Denmark",Hamlet,0,1910,\N,\N,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,6.8,471,Manoharam,US,0,movie,Manoharam,Manoharam,0,2019,\N,122,"Comedy,Drama"
tt9909228,5.8,20,Ximei,US,0,movie,Ximei,Ximei,0,2019,\N,98,Documentary
tt9913056,7.0,21,Swarm Season,US,0,movie,Swarm Season,Swarm Season,0,2019,\N,86,Documentary
tt9913660,5.3,35,No Apology,US,0,movie,No Apology,No Apology,0,2019,\N,102,Drama


In [29]:
df.drop(columns=['endYear'], inplace = True)
df

C:\Users\zbtin\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,averageRating,numVotes,title,region,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0000009,5.9,154,Miss Jerry,US,0,movie,Miss Jerry,Miss Jerry,0,1894,45,Romance
tt0000147,5.2,355,The Corbett-Fitzsimmons Fight,US,0,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,20,"Documentary,News,Sport"
tt0000630,3.2,12,Hamlet,US,0,movie,Hamlet,Amleto,0,1908,\N,Drama
tt0000679,5.2,37,The Fairylogue and Radio-Plays,US,0,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"
tt0000886,5.1,21,"Hamlet, Prince of Denmark",US,0,movie,"Hamlet, Prince of Denmark",Hamlet,0,1910,\N,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,6.8,471,Manoharam,US,0,movie,Manoharam,Manoharam,0,2019,122,"Comedy,Drama"
tt9909228,5.8,20,Ximei,US,0,movie,Ximei,Ximei,0,2019,98,Documentary
tt9913056,7.0,21,Swarm Season,US,0,movie,Swarm Season,Swarm Season,0,2019,86,Documentary
tt9913660,5.3,35,No Apology,US,0,movie,No Apology,No Apology,0,2019,102,Drama


Let's remove the columns that we have multiple values and those that we have used for filtering, so removing directors, titleType and more

In [30]:
df.drop(columns = ['region', 'titleType'], inplace = True)
df

C:\Users\zbtin\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,averageRating,numVotes,title,isOriginalTitle,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0000009,5.9,154,Miss Jerry,0,Miss Jerry,Miss Jerry,0,1894,45,Romance
tt0000147,5.2,355,The Corbett-Fitzsimmons Fight,0,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,20,"Documentary,News,Sport"
tt0000630,3.2,12,Hamlet,0,Hamlet,Amleto,0,1908,\N,Drama
tt0000679,5.2,37,The Fairylogue and Radio-Plays,0,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"
tt0000886,5.1,21,"Hamlet, Prince of Denmark",0,"Hamlet, Prince of Denmark",Hamlet,0,1910,\N,Drama
...,...,...,...,...,...,...,...,...,...,...
tt9906644,6.8,471,Manoharam,0,Manoharam,Manoharam,0,2019,122,"Comedy,Drama"
tt9909228,5.8,20,Ximei,0,Ximei,Ximei,0,2019,98,Documentary
tt9913056,7.0,21,Swarm Season,0,Swarm Season,Swarm Season,0,2019,86,Documentary
tt9913660,5.3,35,No Apology,0,No Apology,No Apology,0,2019,102,Drama


In [61]:
df = df.reset_index().drop_duplicates(subset='index').set_index('index')

In [62]:
genre = df['genres'].str.get_dummies(sep=',')
genre

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,\N
index,,,,,,,,,,,,,,,,,,,,,
tt0000009,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
tt0000147,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
tt0000630,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
tt0000679,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000886,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
tt9909228,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
tt9913056,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
genre = genre.iloc[:, :-]
genre

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
index,,,,,,,,,,,,,,,,,,,,,
tt0000009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
tt0000147,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
tt0000630,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
tt0000679,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000886,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
tt9909228,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
tt9913056,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
years = pd.get_dummies(df.startYear)
years =  years.iloc[:, :-1]
years

,1894,1897,1908,1909,1910,1911,1912,1913,1914,1915,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
index,,,,,,,,,,,,,,,,,,,,,
tt0000009,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000147,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000630,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000679,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000886,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
tt9909228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
tt9913056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [65]:
y = df.iloc[:, 0].values
y

array([5.9, 5.2, 3.2, ..., 7. , 5.3, 8.2])

In [66]:
X = df[['numVotes', 'isOriginalTitle', 'isAdult', 'runtimeMinutes']]
X = X.join(genre)
X = X.join(years)
X

,numVotes,isOriginalTitle,isAdult,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
index,,,,,,,,,,,,,,,,,,,,,
tt0000009,154,0,0,45,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000147,355,0,0,20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000630,12,0,0,\N,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000679,37,0,0,120,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000886,21,0,0,\N,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,471,0,0,122,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
tt9909228,20,0,0,98,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
tt9913056,21,0,0,86,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [78]:
X.replace('\\N', np.nan, inplace = True)

,numVotes,isOriginalTitle,isAdult,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
index,,,,,,,,,,,,,,,,,,,,,
tt0000009,154,0,0,45,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000147,355,0,0,20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000630,12,0,0,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000679,37,0,0,120,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000886,21,0,0,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,471,0,0,122,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
tt9909228,20,0,0,98,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
tt9913056,21,0,0,86,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [88]:
X.runtimeMinutes.isna().value_counts()

False    91564
True      6051
Name: runtimeMinutes, dtype: int64

In [98]:
X['runtimeMinutes'] = X['runtimeMinutes'].astype(float)
X.dtypes

numVotes             int64
isOriginalTitle      int64
isAdult              int64
runtimeMinutes     float64
Action               int64
                    ...   
2017                 uint8
2018                 uint8
2019                 uint8
2020                 uint8
2021                 uint8
Length: 235, dtype: object

In [99]:
X.fillna(X.mean(), inplace = True)

,numVotes,isOriginalTitle,isAdult,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
index,,,,,,,,,,,,,,,,,,,,,
tt0000009,154,0,0,45.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000147,355,0,0,20.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000630,12,0,0,91.597615,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000679,37,0,0,120.000000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0000886,21,0,0,91.597615,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,471,0,0,122.000000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
tt9909228,20,0,0,98.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
tt9913056,21,0,0,86.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


We have our datasets

In [100]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [101]:
X_train.shape

(73211, 235)

In [102]:
y_test.shape

(24404,)

In [107]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = pd.DataFrame(sc_X.fit_transform(X_train))
X_test = pd.DataFrame(sc_X.transform(X_test))

## Multiple Linear Regression

In [108]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [109]:
yhat = lr.predict(X_test)

In [110]:
X.shape

(97615, 236)

In [111]:
import statsmodels.regression.linear_model as sm
X = np.append(arr = np.ones((97615, 1)).astype(int), values = X, axis=1)
X = pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,227,228,229,230,231,232,233,234,235,236
0,1.0,1.0,154.0,0.0,0.0,45.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,355.0,0.0,0.0,20.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,12.0,0.0,0.0,91.597615,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,37.0,0.0,0.0,120.000000,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,21.0,0.0,0.0,91.597615,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
X_opt = X
statreg = sm.OLS(endog = y, exog = X_opt).fit()
statreg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.250
Method:                 Least Squares   F-statistic:                     139.5
Date:                Thu, 27 Aug 2020   Prob (F-statistic):               0.00
Time:                        13:44:42   Log-Likelihood:            -1.5443e+05
No. Observations:               97615   AIC:                         3.093e+05
Df Residuals:                   97379   BIC:                         3.116e+05
Df Model:                         235                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
0              3.5941      0.241     14.936      0.000       3.122       4.066
1              3.5941      0.241     14.936      0.000       3.122       4.066
2           4.135e-06    8.3e-08     49.810      0.000    3.97e-06     4.3e-06
3             -0.0732      0.590     -0.124      0.901      -1.229       1.082
4             -0.7057      0.117     -6.039      0.000      -0.935      -0.477
5           6.665e-05   2.26e-05      2.946      0.003    2.23e-05       0.000
6             -0.3128      0.012    -25.311      0.000      -0.337      -0.289
7              0.5580      0.120      4.659      0.000       0.323       0.793
8             -0.2106      0.014    -14.561      0.000      -0.239      -0.182
9              0.3356      0.029     11.732      0.000       0.280       0.392
10             0.2269      0.021     10.885      0.000       0.186       0.268
11             0.0157      0.010      1.554      0.120      -0.004       0.036
12            -0.0436      0.013     -3.289      0.001      -0.070      -0.018
13             1.2660      0.014     88.649      0.000       1.238       1.294
14             0.3195      0.010     33.512      0.000       0.301       0.338
15            -0.0499      0.021     -2.379      0.017      -0.091      -0.009
16            -0.0372      0.021     -1.742      0.081      -0.079       0.005
17             0.0964      0.046      2.103      0.035       0.007       0.186
18             3.0673      1.179      2.602      0.009       0.757       5.378
19             0.1531      0.023      6.649      0.000       0.108       0.198
20            -0.9160      0.014    -65.908      0.000      -0.943      -0.889
21             0.1953      0.021      9.206      0.000       0.154       0.237
22             0.1047      0.028      3.795      0.000       0.051       0.159
23             0.0964      0.018      5.360      0.000       0.061       0.132
24            -0.0313      0.062     -0.508      0.612      -0.152       0.090
25            -0.7354      0.295     -2.492      0.013      -1.314      -0.157
26            -0.0128      0.012     -1.035      0.301      -0.037       0.011
27            -0.4910      0.021    -23.848      0.000      -0.531      -0.451
28             0.0244      0.031      0.798      0.425      -0.035       0.084
29            -0.2712      0.681     -0.398      0.690      -1.606       1.063
30            -0.2775      0.014    -20.342      0.000      -0.304      -0.251
31             0.0446      0.026      1.710      0.087      -0.007       0.096
32            -0.0907      0.023     -3.947      0.000      -0.136      -0.046
33            -1.2790      1.273     -1.005      0.315      -3.774       1.216
34            -3.2500      1.275     -2.549      0.011      -5.749      -0.751
35            -3.0312      0.962     -3.150      0.002      -4.918      -1.145
36

In [140]:
pValues = statreg.summary2().tables[1]['P>|t|']

In [141]:
pValues = pValues.sort_values(ascending=False)
pValues = pValues.reset_index()
pValues.columns = ['features', 'p-value']

In [154]:
val = pValues.iloc[0, 0]

In [160]:
test = X_opt.drop(columns = [val])
test

,0,1,2,3,4,5,6,7,8,9,...,227,228,229,230,231,232,233,234,235,236
0,1.0,1.0,154.0,0.0,0.0,45.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,355.0,0.0,0.0,20.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,12.0,0.0,0.0,91.597615,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,37.0,0.0,0.0,120.000000,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,21.0,0.0,0.0,91.597615,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97610,1.0,1.0,471.0,0.0,0.0,122.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
97611,1.0,1.0,20.0,0.0,0.0,98.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
97612,1.0,1.0,21.0,0.0,0.0,86.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
97613,1.0,1.0,35.0,0.0,0.0,102.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


107